# 多代理辩论

多代理辩论是一种多代理设计模式，它模拟多轮交互，在每一轮中，代理之间交换他们的响应，并根据其他代理的响应改进自己的响应。

这个示例展示了多代理辩论模式的实现，用于解决来自 [GSM8K 基准测试](https://huggingface.co/datasets/openai/gsm8k)的数学问题。

这种模式中有两种类型的代理：求解代理和聚合代理。求解代理按照[Improving Multi-Agent Debate with Sparse Communication Topology](https://arxiv.org/abs/2406.11776)中描述的技术以稀疏方式连接。求解代理负责解决数学问题并与其他代理交换响应。聚合代理负责将数学问题分发给求解代理，等待他们的最终响应，并聚合响应以获得最终答案。

该模式的工作方式如下：
1. 用户向聚合代理发送数学问题。
2. 聚合代理将问题分发给求解代理。
3. 每个求解代理处理问题，并向其邻居发布响应。
4. 每个求解代理使用来自其邻居的响应来改进其响应，并发布新的响应。
5. 重复步骤 4 固定轮数。在最后一轮中，每个求解代理发布最终响应。
6. 聚合代理使用多数投票来聚合所有求解代理的最终响应以获得最终答案，并发布答案。

我们将使用广播 API，即 {py:meth}`~autogen_core.base.BaseAgent.publish_message`，并且我们将使用主题和订阅来实现通信拓扑。阅读[Topics and Subscriptions](../core-concepts/topic-and-subscription.md)以了解它们的工作原理。

In [38]:
import re
from dataclasses import dataclass
from typing import Dict, List

from autogen_core.application import SingleThreadedAgentRuntime
from autogen_core.base import MessageContext
from autogen_core.components import DefaultTopicId, RoutedAgent, TypeSubscription, default_subscription, message_handler
from autogen_core.components.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    OpenAIChatCompletionClient,
    SystemMessage,
    UserMessage,
)

## 消息协议

首先，我们定义代理使用的消息。
`IntermediateSolverResponse` 是求解代理在每轮中交换的消息，而 `FinalSolverResponse` 是求解代理在最后一轮发布的消息。

In [39]:
@dataclass
class Question:
    content: str


@dataclass
class Answer:
    content: str


@dataclass
class SolverRequest:
    content: str
    question: str


@dataclass
class IntermediateSolverResponse:
    content: str
    question: str
    answer: str
    round: int


@dataclass
class FinalSolverResponse:
    answer: str

## 求解代理

求解代理负责解决数学问题并与其他求解代理交换响应。在收到 `SolverRequest` 后，求解代理使用 LLM 生成答案。然后，它根据轮数发布 `IntermediateSolverResponse` 或 `FinalSolverResponse`。

求解代理被赋予一个主题类型，用于指示代理应该向其发布中间响应的主题。这个主题由其邻居订阅以接收来自该代理的响应 —— 我们稍后将展示如何实现这一点。

我们使用 {py:meth}`~autogen_core.components.default_subscription` 让求解代理订阅默认主题，该主题由聚合代理用来收集求解代理的最终响应。

In [40]:
@default_subscription
class MathSolver(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient, topic_type: str, num_neighbors: int, max_round: int) -> None:
        super().__init__("A debator.")
        self._topic_type = topic_type
        self._model_client = model_client
        self._num_neighbors = num_neighbors
        self._history: List[LLMMessage] = []
        self._buffer: Dict[int, List[IntermediateSolverResponse]] = {}
        self._system_messages = [
            SystemMessage(
                (
                    "You are a helpful assistant with expertise in mathematics and reasoning. "
                    "Your task is to assist in solving a math reasoning problem by providing "
                    "a clear and detailed solution. Limit your output within 100 words, "
                    "and your final answer should be a single numerical number, "
                    "in the form of {{answer}}, at the end of your response. "
                    "For example, 'The answer is {{42}}.'"
                )
            )
        ]
        self._round = 0
        self._max_round = max_round

    @message_handler
    async def handle_request(self, message: SolverRequest, ctx: MessageContext) -> None:
        # Add the question to the memory.
        self._history.append(UserMessage(content=message.content, source="user"))
        # Make an inference using the model.
        model_result = await self._model_client.create(self._system_messages + self._history)
        assert isinstance(model_result.content, str)
        # Add the response to the memory.
        self._history.append(AssistantMessage(content=model_result.content, source=self.metadata["type"]))
        print(f"{'-'*80}\nSolver {self.id} round {self._round}:\n{model_result.content}")
        # Extract the answer from the response.
        match = re.search(r"\{\{(\-?\d+(\.\d+)?)\}\}", model_result.content)
        if match is None:
            raise ValueError("The model response does not contain the answer.")
        answer = match.group(1)
        # Increment the counter.
        self._round += 1
        if self._round == self._max_round:
            # If the counter reaches the maximum round, publishes a final response.
            await self.publish_message(FinalSolverResponse(answer=answer), topic_id=DefaultTopicId())
        else:
            # Publish intermediate response to the topic associated with this solver.
            await self.publish_message(
                IntermediateSolverResponse(
                    content=model_result.content,
                    question=message.question,
                    answer=answer,
                    round=self._round,
                ),
                topic_id=DefaultTopicId(type=self._topic_type),
            )

    @message_handler
    async def handle_response(self, message: IntermediateSolverResponse, ctx: MessageContext) -> None:
        # Add neighbor's response to the buffer.
        self._buffer.setdefault(message.round, []).append(message)
        # Check if all neighbors have responded.
        if len(self._buffer[message.round]) == self._num_neighbors:
            print(
                f"{'-'*80}\nSolver {self.id} round {message.round}:\nReceived all responses from {self._num_neighbors} neighbors."
            )
            # Prepare the prompt for the next question.
            prompt = "These are the solutions to the problem from other agents:\n"
            for resp in self._buffer[message.round]:
                prompt += f"One agent solution: {resp.content}\n"
            prompt += (
                "Using the solutions from other agents as additional information, "
                "can you provide your answer to the math problem? "
                f"The original math problem is {message.question}. "
                "Your final answer should be a single numerical number, "
                "in the form of {{answer}}, at the end of your response."
            )
            # Send the question to the agent itself to solve.
            await self.send_message(SolverRequest(content=prompt, question=message.question), self.id)
            # Clear the buffer.
            self._buffer.pop(message.round)

## 聚合代理

聚合代理负责处理用户问题并将数学问题分发给求解代理。

聚合代理使用 {py:meth}`~autogen_core.components.default_subscription` 订阅默认主题。默认主题用于接收用户问题，接收来自求解代理的最终响应，并将最终答案发布回用户。

在更复杂的应用中，当您想要将多代理辩论隔离为一个子组件时，您应该使用 {py:meth}`~autogen_core.components.type_subscription` 为聚合代理-求解代理通信设置特定的主题类型，并让求解代理和聚合代理都发布和订阅该主题类型。

In [41]:
@default_subscription
class MathAggregator(RoutedAgent):
    def __init__(self, num_solvers: int) -> None:
        super().__init__("Math Aggregator")
        self._num_solvers = num_solvers
        self._buffer: List[FinalSolverResponse] = []

    @message_handler
    async def handle_question(self, message: Question, ctx: MessageContext) -> None:
        print(f"{'-'*80}\nAggregator {self.id} received question:\n{message.content}")
        prompt = (
            f"Can you solve the following math problem?\n{message.content}\n"
            "Explain your reasoning. Your final answer should be a single numerical number, "
            "in the form of {{answer}}, at the end of your response."
        )
        print(f"{'-'*80}\nAggregator {self.id} publishes initial solver request.")
        await self.publish_message(SolverRequest(content=prompt, question=message.content), topic_id=DefaultTopicId())

    @message_handler
    async def handle_final_solver_response(self, message: FinalSolverResponse, ctx: MessageContext) -> None:
        self._buffer.append(message)
        if len(self._buffer) == self._num_solvers:
            print(f"{'-'*80}\nAggregator {self.id} received all final answers from {self._num_solvers} solvers.")
            # Find the majority answer.
            answers = [resp.answer for resp in self._buffer]
            majority_answer = max(set(answers), key=answers.count)
            # Publish the aggregated response.
            await self.publish_message(Answer(content=majority_answer), topic_id=DefaultTopicId())
            # Clear the responses.
            self._buffer.clear()
            print(f"{'-'*80}\nAggregator {self.id} publishes final answer:\n{majority_answer}")

## 设置辩论

我们现在将设置一个包含 4 个求解代理和 1 个聚合代理的多代理辩论。求解代理将以稀疏方式连接，如下图所示：

```
A --- B
|     |
|     |
C --- D
```

每个求解代理与其他两个求解代理相连。例如，代理 A 与代理 B 和 C 相连。

让我们首先创建一个运行时并注册代理类型。

In [42]:
runtime = SingleThreadedAgentRuntime()
await MathSolver.register(
    runtime,
    "MathSolverA",
    lambda: MathSolver(
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
        topic_type="MathSolverA",
        num_neighbors=2,
        max_round=3,
    ),
)
await MathSolver.register(
    runtime,
    "MathSolverB",
    lambda: MathSolver(
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
        topic_type="MathSolverB",
        num_neighbors=2,
        max_round=3,
    ),
)
await MathSolver.register(
    runtime,
    "MathSolverC",
    lambda: MathSolver(
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
        topic_type="MathSolverC",
        num_neighbors=2,
        max_round=3,
    ),
)
await MathSolver.register(
    runtime,
    "MathSolverD",
    lambda: MathSolver(
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
        topic_type="MathSolverD",
        num_neighbors=2,
        max_round=3,
    ),
)
await MathAggregator.register(runtime, "MathAggregator", lambda: MathAggregator(num_solvers=4))

AgentType(type='MathAggregator')

现在我们将使用 {py:class}`~autogen_core.components.TypeSubscription` 创建求解代理拓扑，它将每个求解代理的发布主题类型映射到其邻居的代理类型。

In [43]:
# Subscriptions for topic published to by MathSolverA.
await runtime.add_subscription(TypeSubscription("MathSolverA", "MathSolverD"))
await runtime.add_subscription(TypeSubscription("MathSolverA", "MathSolverB"))

# Subscriptions for topic published to by MathSolverB.
await runtime.add_subscription(TypeSubscription("MathSolverB", "MathSolverA"))
await runtime.add_subscription(TypeSubscription("MathSolverB", "MathSolverC"))

# Subscriptions for topic published to by MathSolverC.
await runtime.add_subscription(TypeSubscription("MathSolverC", "MathSolverB"))
await runtime.add_subscription(TypeSubscription("MathSolverC", "MathSolverD"))

# Subscriptions for topic published to by MathSolverD.
await runtime.add_subscription(TypeSubscription("MathSolverD", "MathSolverC"))
await runtime.add_subscription(TypeSubscription("MathSolverD", "MathSolverA"))

# All solvers and the aggregator subscribe to the default topic.

## 解决数学问题

现在让我们运行辩论来解决一个数学问题。我们向默认主题发布一个 `SolverRequest`，聚合代理将开始辩论。

In [44]:
question = "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"
runtime.start()
await runtime.publish_message(Question(content=question), DefaultTopicId())
await runtime.stop_when_idle()

--------------------------------------------------------------------------------
Aggregator MathAggregator:default received question:
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
--------------------------------------------------------------------------------
Aggregator MathAggregator:default publishes initial solver request.
--------------------------------------------------------------------------------
Solver MathSolverC:default round 0:
In April, Natalia sold 48 clips. In May, she sold half as many, which is 48 / 2 = 24 clips. To find the total number of clips sold in April and May, we add the amounts: 48 (April) + 24 (May) = 72 clips. 

Thus, the total number of clips sold by Natalia is {{72}}.
--------------------------------------------------------------------------------
Solver MathSolverB:default round 0:
In April, Natalia sold 48 clips. In May, she sold half as many 